This is about running ABMs with COmplexVID-19

In [9]:
import sys
sys.path.append("/src/pyciemss/utils/complexvid_scabini")
from pyciemss.utils.complexvid_scabini import comunities
from pyciemss.utils.complexvid_scabini import model
from pyciemss.utils.complexvid_scabini import dynamics_OTM